In [1]:
    import numpy as np
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt
    import matplotlib
    %matplotlib inline

    import findspark
    findspark.init()

    # Import and start a Spark session.
    from pyspark.sql import *
    import pyspark.sql.functions as F

    spark = SparkSession.builder.getOrCreate()

/home/pierre/.local/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
def load_insights(months):
    return {month: spark.read.parquet('hashtags_insights_{}.parquet'.format(month)) for month in months} 

We start by loading the different hashtags insights for each months. The month of January is omitted as it is corrupted, the month of June is also omitted as it is too small (we only have data for an extract of two days), and the month december does not exists.

In [3]:
hashtags = load_insights(['02', '03', '04', '05', '07', '08', '09', '10', '11'])
#hashtags = load_insights(['09', '10', '11'])

For reasons we didn't understood, our hashtags insights for the month of augusts contained some duplicates rows. Hence, we remove them before continuing our analysis.

In [4]:
hashtags['08'] = hashtags['08'].dropDuplicates()

In [5]:
all_days = [str(i) for i in range(1, 32)]
all_months = [str(i).zfill(2) for i in range(2, 13)]

In [6]:
allColumns = \
    ["tag", "count", "print"] + \
    [month + "_print_" + day for day in all_days for month in all_months] + \
    [month + "_count_" + day for day in all_days for month in all_months]

In [7]:
def rename_columns(insights, prefix):
    printCols = [c for c in insights.columns if "print_" in c]
    nameCols = [c for c in insights.columns if "count_" in c]
    print(insights, prefix)
    for x in printCols + nameCols:
        if insights.where(F.col(x) != 0).count() == 0:
            insights = insights.drop(x)
        else:
            insights = insights.withColumnRenamed(x, prefix + "_" + x)
    return insights.drop("count").drop("print")

In [8]:
insights = [rename_columns(hashtags[key], key) for key in hashtags]

DataFrame[tag: string, count: bigint, print: bigint, print_1: bigint, print_2: bigint, print_3: bigint, print_4: bigint, print_5: bigint, print_6: bigint, print_7: bigint, print_8: bigint, print_9: bigint, print_10: bigint, print_11: bigint, print_12: bigint, print_13: bigint, print_14: bigint, print_15: bigint, print_16: bigint, print_17: bigint, print_18: bigint, print_19: bigint, print_20: bigint, print_21: bigint, print_22: bigint, print_23: bigint, print_24: bigint, print_25: bigint, print_26: bigint, print_27: bigint, print_28: bigint, print_29: bigint, print_30: bigint, print_31: bigint, count_1: bigint, count_2: bigint, count_3: bigint, count_4: bigint, count_5: bigint, count_6: bigint, count_7: bigint, count_8: bigint, count_9: bigint, count_10: bigint, count_11: bigint, count_12: bigint, count_13: bigint, count_14: bigint, count_15: bigint, count_16: bigint, count_17: bigint, count_18: bigint, count_19: bigint, count_20: bigint, count_21: bigint, count_22: bigint, count_23: b

DataFrame[tag: string, count: bigint, print: bigint, print_1: bigint, print_2: bigint, print_3: bigint, print_4: bigint, print_5: bigint, print_6: bigint, print_7: bigint, print_8: bigint, print_9: bigint, print_10: bigint, print_11: bigint, print_12: bigint, print_13: bigint, print_14: bigint, print_15: bigint, print_16: bigint, print_17: bigint, print_18: bigint, print_19: bigint, print_20: bigint, print_21: bigint, print_22: bigint, print_23: bigint, print_24: bigint, print_25: bigint, print_26: bigint, print_27: bigint, print_28: bigint, print_29: bigint, print_30: bigint, print_31: bigint, count_1: bigint, count_2: bigint, count_3: bigint, count_4: bigint, count_5: bigint, count_6: bigint, count_7: bigint, count_8: bigint, count_9: bigint, count_10: bigint, count_11: bigint, count_12: bigint, count_13: bigint, count_14: bigint, count_15: bigint, count_16: bigint, count_17: bigint, count_18: bigint, count_19: bigint, count_20: bigint, count_21: bigint, count_22: bigint, count_23: b

In [9]:
from functools import reduce

def joinAll(dfs):
    return reduce(lambda prev, cur: cur.join(prev, on='tag', how='outer'), dfs)
unifiedInsights = joinAll(insights)

In [10]:
#unifiedInsights.write.mode("overwrite").parquet("unifiedinsights.parquet")

KeyboardInterrupt: 

In [ ]:
#unifiedInsights = spark.read.parquet("unifiedinsights.parquet")

In [11]:
unifiedInsights = unifiedInsights.fillna(0).cache()

In [12]:
count_columns = [c for c in unifiedInsights.columns if "count" in c]
print_columns = [c for c in unifiedInsights.columns if "print" in c]

In [13]:
unifiedInsights = unifiedInsights.withColumn("count", sum(F.col(c) for c in count_columns))
unifiedInsights = unifiedInsights.withColumn("print", sum(F.col(c) for c in print_columns))

In [14]:
unifiedInsights.write.mode("overwrite").parquet("hashtags-insights.parquet")

In [15]:
len(count_columns)

264

In [16]:
len(print_columns)

264

In [17]:
insights = spark.read.parquet("hashtags-insights.parquet")

In [18]:
unifiedInsights.select(F.col('tag')).count()

15177260

In [45]:
insights.select(F.col('tag')).show()

+-----------+
|        tag|
+-----------+
|000bloodrun|
|  005GOBLIN|
|  005GOBLIN|
|  005GOBLIN|
|     007E8C|
|     010tml|
|     010tml|
|     010tml|
|       016s|
|      01트친소|
|   024_Full|
|      02AGO|
|      02AGO|
|      02AGO|
|      02May|
|    02년생입니당|
|     0309상암|
|03Kasım2002|
|   040zwemt|
|    04Junio|
+-----------+
only showing top 20 rows



We define a function which will allow us to unify the columns of every sub dataframes by renaming some columns and adding some others.

In [ ]:
def rename_columns(insights, prefix):
    printCols = [c for c in insights.columns if "print_" in c]
    nameCols = [c for c in insights.columns if "count_" in c]
    print(insights, prefix)
    for x in printCols + nameCols:
        insights = insights.withColumnRenamed(x, prefix + "_" + x)
    columns = insights.columns
    for i, x in enumerate(allColumns):
        if i % 50 == 0:
            print(i, len(allColumns))
        if not x in columns:
            insights = insights.withColumn(x, F.lit(0).cast("long"))
    return insights

We then apply this function to all of our insights.

In [ ]:
insights = [rename_columns(hashtags[key], key) for key in hashtags]

DataFrame[tag: string, count: bigint, print: bigint, print_1: bigint, print_2: bigint, print_3: bigint, print_4: bigint, print_5: bigint, print_6: bigint, print_7: bigint, print_8: bigint, print_9: bigint, print_10: bigint, print_11: bigint, print_12: bigint, print_13: bigint, print_14: bigint, print_15: bigint, print_16: bigint, print_17: bigint, print_18: bigint, print_19: bigint, print_20: bigint, print_21: bigint, print_22: bigint, print_23: bigint, print_24: bigint, print_25: bigint, print_26: bigint, print_27: bigint, print_28: bigint, print_29: bigint, print_30: bigint, print_31: bigint, count_1: bigint, count_2: bigint, count_3: bigint, count_4: bigint, count_5: bigint, count_6: bigint, count_7: bigint, count_8: bigint, count_9: bigint, count_10: bigint, count_11: bigint, count_12: bigint, count_13: bigint, count_14: bigint, count_15: bigint, count_16: bigint, count_17: bigint, count_18: bigint, count_19: bigint, count_20: bigint, count_21: bigint, count_22: bigint, count_23: b

We merge every insights in one unique dataframe. We have to make sure that every columns is in the same order.

In [ ]:
from functools import reduce

def unionAll(dfs):
    return reduce(DataFrame.union, dfs)
unifiedInsights = unionAll([i.select(allColumns) for i in insights])

As some hashtags might appear in different months, we need to merge duplicates occurencies of the same hashtags.

In [ ]:
columnsToSum = [x for x in unifiedInsights.columns if x != "tag"]
aggExpressions = [F.sum(F.col(col)).alias(col) for col in columnsToSum]

In [ ]:
unifiedInsightsGB = unifiedInsights.groupBy('tag').agg(*aggExpressions)

In [ ]:
unifiedInsightsGB.write.mode("overwrite").parquet("unifiedinsights.parquet")

In [ ]:
unifiedInsights = spark.read.parquet("unifiedinsights.parquet")

In [ ]:
unifiedInsights.where(F.col('11_print_4') != 0).count()

In [ ]:
hashtags["11"].where(F.col("print_4") != 0).count()

In [ ]:
for x in allColumns:
    if unifiedInsights.where(F.col(x) != 0).count() == 0:
        unifiedInsights = unifiedInsights.drop(x)

In [ ]:
[x for x in unifiedInsights.columns if "11" in x]

In [ ]:
unifiedInsights.columns

In [ ]:
[x for x in unifiedInsights.columns if "01" in x]

In [ ]:
unifiedInsights.write.mode("overwrite").parquet("insights.parquet")

In [ ]:
unifiedInsights.where(F.col('03_count_1') != 0).count()


In [ ]:
hashtags["03"].columns